<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 다룰 내용

- 무엇을 해볼까요?

In [8]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config InlineBackend.figure_format='retina'

# 1. 데이터베이스 연결

In [5]:
# 라이브러리 설치
!pip install pymysql

In [9]:
# 라이브러리 불러오기
import pymysql

In [10]:
mysql = pymysql.connect(host='20.194.2.187',
                       port=3306,
                       user='Student',
                       password='Pa$$w0rd',
                       database='titanic')

# 2. 데이터 가져오기    

In [11]:
# 쿼리문
sql = """
select Survived, Pclass, Sex,
                 ifnull(Age, (select avg(Age) from titanic)) as Age, 
                 SibSp + Parch as FamilyNo,
                 Fare, 
                 ifnull(Embarked, 'S') as Embarked
from titanic
"""

In [12]:
# 데이터 읽어오기
data = pd.read_sql(sql, con=mysql)

In [13]:
# 확인
data.head()

,Survived,Pclass,Sex,Age,FamilyNo,Fare,Embarked
0,0,3,male,22.0,1,7.2500,S
1,1,1,female,38.0,1,71.2833,C
2,1,3,female,26.0,0,7.9250,S
3,1,1,female,35.0,1,53.1000,S
4,0,3,male,35.0,0,8.0500,S


In [14]:
data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
FamilyNo    0
Fare        0
Embarked    0
dtype: int64

# 3. 데이터 준비

In [15]:
# 가변수화
cols = ['Pclass', 'Sex', 'Embarked']
data = pd.get_dummies(data, columns=cols, drop_first=True)

#확인
data.head()

,Survived,Age,FamilyNo,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0,22.0,1,7.2500,0,1,1,0,1
1,1,38.0,1,71.2833,0,0,0,0,0
2,1,26.0,0,7.9250,0,1,0,0,1
3,1,35.0,1,53.1000,0,0,0,0,1
4,0,35.0,0,8.0500,0,1,1,0,1


In [16]:
# x, y 분리
target = 'Survived'
x = data.drop(target, axis=1)
y = data.loc[:, target]

In [17]:
# 정규화
x = (x - x.min()) / (x.max() - x.min())

# 확인
x

,Age,FamilyNo,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0.275000,0.1,0.014151,0.0,1.0,1.0,0.0,1.0
1,0.475000,0.1,0.139136,0.0,0.0,0.0,0.0,0.0
2,0.325000,0.0,0.015469,0.0,1.0,0.0,0.0,1.0
3,0.437500,0.1,0.103644,0.0,0.0,0.0,0.0,1.0
4,0.437500,0.0,0.015713,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...
886,0.337500,0.0,0.025374,1.0,0.0,1.0,0.0,1.0
887,0.237500,0.0,0.058556,0.0,0.0,0.0,0.0,1.0
888,0.371411,0.3,0.045771,0.0,1.0,0.0,0.0,1.0
889,0.325000,0.0,0.058556,0.0,0.0,1.0,0.0,0.0


In [18]:
# 학습용, 평가용 분리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4. 모델링 

In [19]:
#1단계 : 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [20]:
# 2단계 : 선언하기
model = KNeighborsClassifier()

In [21]:
# 3단계 : 학습하기
model.fit(x_train,y_train)

KNeighborsClassifier()

In [22]:
# 4단계 : 예측하기
y_pred = model.predict(x_test)

In [23]:
# 5단계 : 평가하기
accuracy_score(y_test, y_pred)

0.7649253731343284